In [22]:
# activate the venv and pip install first
from dotenv import load_dotenv
load_dotenv()
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json

print(os.getenv("OPENAI_API_KEY"))
print(os.getenv("PINECONE_API_KEY"))


sk-M26pYEjTlXwOhP2zpRx5upcaKiHEicPu-z6I-r88q1T3BlbkFJmqMQHU6UJhN6i37a0eteNVPp2OGXj3FQNg4390zCcA
84f7936f-fe8b-4a5b-8fe8-5bbb07bc8ecb


In [23]:
# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [ ]:
# Create a Pinecone index
# run again make error; can't create same index
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [24]:
# Load the review data
data = json.load(open("reviews.json"))

processed_data = []
client = OpenAI()

data['reviews']

[{'professor': 'Dr. Emily Sanders',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Dr. Sanders is incredibly knowledgeable and passionate about biology. Her lectures are engaging and well-structured.'},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Prof. Chen is an outstanding teacher. He explains complex concepts clearly and is always available for extra help.'},
 {'professor': 'Dr. Sarah Johnson',
  'subject': 'Psychology',
  'stars': 3,
  'review': "Dr. Johnson's course content is interesting, but her lectures can be a bit disorganized at times."},
 {'professor': 'Prof. Robert Williams',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Prof. Williams makes math enjoyable. His problem-solving techniques are very helpful.'},
 {'professor': 'Dr. Amanda Lee',
  'subject': 'Chemistry',
  'stars': 5,
  'review': "Dr. Lee's enthusiasm for chemistry is contagious. Her lab sessions are particularly well-designed and informative."},

In [25]:
# Create embeddings for each review
for review in data["reviews"]:
    response = client.embeddings.create(
        input=review['review'], model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append(
        {
            "values": embedding,
            "id": review["professor"],
            "metadata":{
                "review": review["review"],
                "subject": review["subject"],
                "stars": review["stars"],
            }
        }
    )

In [26]:
processed_data[0]

{'values': [0.025389742,
  -0.011489499,
  0.017554827,
  0.054216076,
  0.0048150746,
  -0.0014233644,
  0.016298164,
  0.029134087,
  -0.007212995,
  0.024569064,
  0.009604503,
  -0.025299981,
  -0.011880605,
  -0.008738944,
  -0.0156185385,
  0.029005857,
  -0.03877706,
  -0.042136714,
  0.004924071,
  -0.006924475,
  0.07011672,
  -0.008879998,
  0.046034936,
  -0.015990408,
  -0.02509481,
  -0.025158927,
  0.04824051,
  0.013797658,
  -0.011778019,
  0.0057800133,
  0.07263005,
  -0.014502929,
  -0.002255263,
  -0.015798062,
  -0.032519385,
  0.043470316,
  -0.027723547,
  0.01268846,
  0.019939924,
  0.03013429,
  0.026569467,
  0.004353443,
  -0.047855817,
  0.030057352,
  0.039956786,
  -0.024722941,
  -0.030493338,
  -0.02810824,
  -0.011495911,
  0.0154903075,
  -0.032416802,
  0.015631361,
  0.026825929,
  -0.009021052,
  -0.022594307,
  0.059960827,
  -0.010027666,
  0.014772214,
  -0.030416397,
  -0.036135502,
  0.07052706,
  0.020786248,
  0.018901253,
  0.005969154,
  -

In [27]:
# Insert the embeddings into the Pinecone index
index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

Upserted count: 20


In [28]:
# Print index statistics
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}
